In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

In [ ]:
G = nx.gnp_random_graph(100, 0.5, directed=False)

In [ ]:
degrees = []
for node in G.nodes:
    degrees.append(G.degree(node))

In [ ]:
fit_params = stats.powerlaw.fit(degrees)
print(fit_params)

In [ ]:
plt.figure(figsize=(7,4))
x = np.linspace(0, max(degrees), 100)
pdff = stats.powerlaw.pdf(x, *fit_params)
plt.plot(x, pdff, 'k-', lw=2, label='Power law fit')
plt.hist(degrees, bins='auto', histtype='stepfilled', density=True, alpha=0.2, label='Degree distribution')

In [ ]:
def kld(p,q):
    assert len(p) == len(q)
    # assume they are defined on the same sample space
    kl = 0
    for ii in range(len(p)):
        if p[ii] > 0 and q[ii] > 0:
            kl += p[ii] * np.log(p[ii]/q[ii])
    return kl

In [ ]:
degrees_normalized = (degrees - fit_params[1]) / fit_params[2]
hist, bins = np.histogram(degrees_normalized, 
                          bins=25, 
                          #range=(0, np.max(degrees_normalized)), 
                          density=True)
bins = bins[0:-1]
# shifted
plt.figure(figsize=(7,4))
x = bins
pdff = stats.powerlaw.pdf(x, fit_params[0])
kl = kld(hist, pdff)
plt.plot(x, pdff, 'k-', lw=2, label='Power law fit')
plt.stem(bins, hist, label='Empirical')
plt.title('a=%0.03f KLD=%0.03f' % (fit_params[0], kl))
plt.legend()

In [ ]:
min(degrees_normalized)

In [ ]:
stats.powerlaw.pdf(min(degrees_normalized), fit_params[0])

In [ ]:
np.log(0)

In [ ]:
# try the method outlined here:https://lordgrilo.github.io/complexity-book/2-networkx/nb04_powerlaw_fitting.html#the-powerlaw-package

from collections import Counter 
deg=dict(G.degree()).values()
deg_distri=Counter(deg)

In [ ]:
x=[]
y=[]
for i in sorted(deg_distri):   
    x.append(i)
    y.append(deg_distri[i]/len(G))


plt.figure(figsize=(6,3))
plt.hist(deg, bins=90, density=True, log=True, histtype='stepfilled')

plt.plot(x,y,'black','o')
plt.xscale('log')
plt.yscale('log')
plt.xticks(fontsize=20)
plt.yticks(fontsize=22)
plt.xlabel('degree $k$', fontsize=22)
plt.ylabel('$P(k)$', fontsize=22)

In [ ]:
import powerlaw as pwl
degree=list(deg)
pwl_distri=pwl.pdf(degree, bins=90)

In [ ]:
plt.plot(pwl_distri[0][0:-1],pwl_distri[1],'r--',linewidth=2)

In [ ]:
fo = pwl.Fit(pwl_distri[1])

In [ ]:
fo.power_law.KS()